# Proyecto 1

In [ ]:
import os
import glob

In [ ]:
import cv2
import numpy as np

In [ ]:
def stitch_images(images, keypoints, descriptors, homographies):
    # Crear una imagen vacía con el tamaño necesario para acomodar todas las imágenes alineadas
    height, width, _ = images[0].shape
    stitched_img = np.zeros((height, sum([img.shape[1] for img in images]), 3), dtype=np.uint8)

    # Colocar la primera imagen en la imagen combinada
    stitched_img[0:height, 0:width] = images[0]

    # Aplicar las homografías para cada imagen y colocarla en la imagen combinada
    for i in range(1, len(images)):
        # Aplicar la transformación a la imagen
        transformed_img = cv2.warpPerspective(images[i], np.linalg.inv(homographies[i-1]), (width * (i+1), height))

        # Encontrar la región no negra de la imagen transformada
        mask = np.all(transformed_img != [0, 0, 0], axis=-1)

        # Combinar la imagen transformada con la imagen combinada
        stitched_img[mask] = transformed_img[mask]

    return stitched_img

In [ ]:
# 1. Leer las imágenes del directorio
path = 'images/*'
images = [cv2.imread(file) for file in glob.glob(path)]

In [ ]:
# 2. Encontrar características únicas en cada imagen
sift = cv2.SIFT_create()

keypoints = []
descriptors = []

for img in images:
    kp, des = sift.detectAndCompute(img, None)
    keypoints.append(kp)
    descriptors.append(des)

In [ ]:
# 3. Realizar un emparejamiento de características entre las imágenes
matcher = cv2.BFMatcher()
matches = []

for i in range(len(images) - 1):
    matches.append(matcher.knnMatch(descriptors[i], descriptors[i+1], k=2))

In [ ]:
# 4. Utilizar la homografía para transformar las imágenes y alinearlas
good_matches = []

for match in matches:
    good = []
    for m, n in match:
        if m.distance < 0.75 * n.distance:
            good.append(m)
    good_matches.append(good)

homographies = []

for i in range(len(good_matches)):
    src_pts = np.float32([keypoints[i][m.queryIdx].pt for m in good_matches[i]]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints[i+1][m.trainIdx].pt for m in good_matches[i]]).reshape(-1, 1, 2)
    M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    homographies.append(M)

In [ ]:
# 5. Combinar las imágenes
result = stitch_images(images, keypoints, descriptors, homographies)

# 6. Guardar y mostrar la imagen panorámica completa
cv2.imwrite('panorama.jpg', result)
cv2.imshow('Panorama', result)
cv2.waitKey(0)
cv2.destroyAllWindows()